# 11.2 The XML Data Format

**XML**, which stands for eXtensible Markup Language, is another way to represent hierarchical data. The basic building block of XML is the **tag**, denoted by angle brackets `<>`.

For example, a data set of movies might be represented using XML as follows:

```
<movies>
  <movie id="1" title="The Godfather">
    <director id="50" name="Coppola, Francis Ford">
    </director>
    <releasedate>1972-03-24</releasedate>
    <character id="100" name="Vito Corleone">
      <actor id="200" name="Brando, Marlon">
      </actor>
    </character>
    <character id="101" name="Michael Corleone">
      <actor id="201" name="Pacino, Al">
      </actor>
    </character>
    ...
  </movie>
  <movie id="2" title="The Godfather: Part II">
    <director id="50" name="Coppola, Francis Ford">
    </director>
    <releasedate>1974-10-20</releasedate>
    <character id="101" name="Michael Corleone">
      <actor id="201" name="Pacino, Al">
      </actor>
    </character>
    <character id="100" name="Vito Corleone">
      <actor id="250" name="De Niro, Robert">
      </actor>
    </character>
    ...
  </movie>
  ...
</movies>
```

Note the following features of XML:

- Every tag `<a>` has a corresponding closing tag `</a>`. You can always recognize a closing tag by the forward slash `/`.
- Additional tags and/or strings can be nested between the opening and closing tags. In the example above, `<actor>` is nested between `<character>` and `</character>`, and `<character>` is nested between `<movie>` and `</movie>`. The nesting is used to represent hierarchy.
- Indentation is used to make the code more readable (to make it easier to see the nesting structure). But it is optional.
- Attributes can be associated with each tag, like `id=` and `name=` with the `<character>` tag and `id=` and `title=` with the `<movie>` tag.

Each tag represents a variable in the data set. Unlike JSON, which uses lists to represent repeated fields, XML represents repeated fields by simply repeating tags where necessary. In the example above, there are multiple instances of `<movie>` within `<movies>` and multiple instances of `<character>` within `<movie>`, so `movie` and `character` are both repeated fields. (In fact, `director` is also a repeated field, but it is impossible to tell from the code above, since the movies shown above only have one director.)

You will learn XML by working with the same New York Philharmonic data as in the previous section, except that the data is now stored in XML format. Let's look at this file on disk.

In [2]:
!ls -l /data301/data/nyphil/

total 78912
-rw-r--r-- 1 root root 35284693 Feb 25 08:35 complete.json
-rw-r--r-- 1 root root 45514892 Feb 25 08:36 complete.xml


Notice that this XML file is nearly twice as large as the JSON file. Although XML is more readable than JSON, it is a more expensive way to store hierarchical data, primarily because of the cost of storing both the opening and closing tags.

There are several libraries in Python for working with XML, including BeautifulSoup (which we will use in the next section to parse HTML), ElementTree, and `lxml`. We will use `lxml` to work with XML data because it is fastest for large data sets, provided that the data is well-formed. The `lxml` library provides a convenient API that replicates all of the functionality of ElementTree, plus implements a few additional features that are useful for data analysis.

In [3]:
from lxml import etree

First, let's read in the data using `lxml`. The `.parse()` function of ElementTree reads in an XML document from a file or URL and returns a tree-like data structure called an ElementTree.

In [4]:
tree = etree.parse("/data301/data/nyphil/complete.xml")

Every XML document has a single "root" tag that encloses all of the other tags. For the New York Philharmonic data, this root tag is `<programs>`.

In [5]:
tree.getroot()

<Element programs at 0x7f0ea45f34c8>

If the XML data is already stored as a string in memory, then we instead use the `.fromstring()` method. Note that `.fromstring()` returns the root tag directly.

In [6]:
with open("/data301/data/nyphil/complete.xml", "rb") as f:
    string = f.read()
    
etree.fromstring(string)

<Element programs at 0x7f0e8d5204c8>

Each direct descendant, or **child**, of `<programs>` is a program. To find the direct descendants of a tag, we call the `.getchildren()` method.

In [7]:
programs = tree.getroot()
print(len(programs.getchildren()))
programs.getchildren()[:10]

14009


[<Element program at 0x7f0e8d507208>,
 <Element program at 0x7f0e8d507188>,
 <Element program at 0x7f0e8d520e08>,
 <Element program at 0x7f0e8d520e48>,
 <Element program at 0x7f0e8d520dc8>,
 <Element program at 0x7f0e8d53f088>,
 <Element program at 0x7f0e8d53f0c8>,
 <Element program at 0x7f0e8d53f108>,
 <Element program at 0x7f0e8d53f148>,
 <Element program at 0x7f0e8d520d88>]

Let's print out the first of these programs. There are two ways to get the first program.

In [8]:
# METHOD 1: Get it from the list above.
program = programs.getchildren()[0]

# METHOD 2: Use .find() to find the first instance of a tag.
program = tree.find("program")
program

<Element program at 0x7f0e8d507208>

Now let's see how the data is represented by printing out the XML of this program. To do this, we use the `etree.tostring()` function.

In [9]:
print(etree.tostring(program, encoding="unicode"))

<program>
        <id>00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1</id>
        <programID>3853</programID>
        <orchestra>New York Philharmonic</orchestra>
        <season>1842-43</season>
        <concertInfo>
            <eventType>Subscription Season</eventType>
            <Location>Manhattan, NY</Location>
            <Venue>Apollo Rooms</Venue>
            <Date>1842-12-07T05:00:00Z</Date>
            <Time>8:00PM</Time>
        </concertInfo>
        <worksInfo>
            <work ID="52446*">
                <composerName>Beethoven,  Ludwig  van</composerName>
                <workTitle>SYMPHONY NO. 5 IN C MINOR, OP.67</workTitle>
                <conductorName>Hill, Ureli Corelli</conductorName>
            </work>
            <work ID="8834*4">
                <composerName>Weber,  Carl  Maria Von</composerName>
                <workTitle>OBERON</workTitle>
                <movement>"Ozean, du Ungeheuer" (Ocean, thou mighty monster), Reiza (Scene and Aria), Act II</movement>


Hopefully, the basic structure of this data is already familiar to you from previous section. "Work", "concertInfo", and "soloist" are repeated fields inside "program". One difference between the JSON and the XML is that "work" is not directly nested within "program"; the "work" tags are all nested inside an additional "worksInfo" tag.

Now suppose that we want to flatten the data at the level of soloists. To get all of the soloists, we can use the `.findall()` method. Let's first try the obvious solution, which does not work:

In [10]:
programs.findall("soloist")

[]

Why did `lxml` fail to find any `<soloist>` tags? That's because `.findall()` only searches among the direct descendants of a tag. We called `.findall()` on the `<programs>` tag, but all of its descendants are `<program>` tags.

To specify that `lxml` should look for `<soloist>` tags among all descendants, not just direct ones, we use the `.xpath()` command, which allows us to specify an XPath expression. [XPath](https://www.w3schools.com/xml/xpath_syntax.asp) is a language used to select nodes from XML documents. The XPath expression to select all descendants named `<soloist>` of the current tag is `".//soloist"`. We pass this expression to the `.xpath()` method.

In [11]:
soloists = programs.xpath("//soloist")
len(soloists)

56931

Now, to flatten the data at the level of soloists, we just need to turn `soloists` into a `DataFrame` with as many rows. But what if we want to include information from parent levels, like the composer of the work the soloist played? There are two ways.

### Method 1

Since `<composerName>` is a descendant of `<work>`, one way is to navigate up to the level of `<work>` by calling `.getparent()` repeatedly and then find `<composerName>` among its descendants:

In [12]:
soloist = soloists[0]

# The first .getparent() returns the <soloists> tag.
# The second .getparent() returns the <work> tag.
# You have to figure this out by inspecting the XML.
work = soloist.getparent().getparent()
work.xpath(".//composerName")

[<Element composerName at 0x7f0e8d1bfb08>]

This is a list with one tag, so we extract that tag and the text inside it.

In [13]:
work.xpath(".//composerName")[0].text

'Weber,  Carl  Maria Von'

### Method 2

As the number of levels of nesting increases, it quickly becomes impractical to call `.getparent()` repeatedly. We want to be able to jump directly to the right ancestor. The easiest way to do this is to use the XPath expression for an ancestor. To search for all ancestors named "work", we can use the XPath expression `"ancestor::work"`.

In [14]:
soloist.xpath("ancestor::work")

[<Element work at 0x7f0e8d1bfa08>]

Now, we can extract this single work tag and find its descendants named `<composerName>`. Or better yet, we can combine this step with the above step into a single XPath expression.

In [15]:
soloist.xpath("ancestor::work//composerName")[0].text

'Weber,  Carl  Maria Von'

Now let's put it all together. We will flatten the data to get a `DataFrame` with one soloist per row. We will keep track of the soloist's name, instrument, and role---as well as the composer of the work they performed. Unfortunately, it is much more manual to do this with XML than with JSON. There is no XML equivalent of the `json_normalize` function that will automatically produce a `DataFrame`, so we have to construct the `DataFrame` ourselves.

In [16]:
import pandas as pd

rows = []

soloists = programs.xpath(".//soloist")
for soloist in soloists:
    row = {}
    row["soloistName"] = soloist.find("soloistName").text
    row["soloistInstrument"] = soloist.find("soloistInstrument").text
    row["soloistRoles"] = soloist.find("soloistRoles").text
    row["composerName"] = soloist.xpath("ancestor::work//composerName")[0].text
    rows.append(row)
    
soloistsdf = pd.DataFrame(rows)
soloistsdf

,composerName,soloistInstrument,soloistName,soloistRoles
0,"Weber, Carl Maria Von",Soprano,"Otto, Antoinette",S
1,"Hummel, Johann",Piano,"Scharfenberg, William",A
2,"Hummel, Johann",Violin,"Hill, Ureli Corelli",A
3,"Hummel, Johann",Viola,"Derwort, G. H.",A
4,"Hummel, Johann",Cello,"Boucher, Alfred",A
5,"Hummel, Johann",Double Bass,"Rosier, F. W.",A
6,"Rossini, Gioachino",Soprano,"Otto, Antoinette",S
7,"Rossini, Gioachino",Tenor,"Horn, Charles Edward",S
8,"Beethoven, Ludwig van",Tenor,"Horn, Charles Edward",S
9,"Mozart, Wolfgang Amadeus",Soprano,"Otto, Antoinette",S


Now, this is a `DataFrame` that we can analyze easily. For example, here is how many times Benny Goodman programmed a work by Mozart with the NY Phil:

In [17]:
soloistsdf[soloistsdf["soloistName"] == "Goodman, Benny"].composerName.value_counts()

Mozart,  Wolfgang  Amadeus      3
Weber,  Carl  Maria Von         3
Gershwin,  George               2
Sauter,  Eddie                  2
Sampson,  Edgar                 1
Copland,  Aaron                 1
Debussy,  Claude                1
Unspecified,                    1
Williams,  Mary Lou             1
Green,  Johnny                  1
Anthem,                         1
Confrey,  Zez                   1
Ellington,  Duke                1
Cannon,  Hughie                 1
Basie,  Count                   1
Baxter,  Phil                   1
Handy,  William  Christopher    1
Prima,  Louis                   1
Youmans,  Vincent               1
Name: composerName, dtype: int64

# RESTful Web Services

Many RESTful web services return data in XML format. Like before, we use the `requests` library in Python to issue the HTTP request. For example, the website [FloatRates](http://www.floatrates.com/feeds.html) provides exchange rates between world currencies in XML format.

In [18]:
import requests
resp = requests.get("http://www.floatrates.com/daily/usd.xml")
resp

<Response [200]>

The XML is stored in the `.content` attribute of the response object. We can parse this string into an ElementTree using the `.fromstring()` function in the `lxml` library. Recall that this returns the root tag of the XML document.

In [19]:
etree.fromstring(resp.content)

<Element channel at 0x7f0ea4684308>

# Exercises

Exercises 1 and 2 deal with the New York Philharmonic data set from above. These exercises are the same as the ones in the previous section, except that now you have to do them with XML.

**Exercise 1.** What is the most frequent start time for New York Philharmonic concerts?

In [25]:
times = programs.xpath("//concertInfo/Time")
pd.Series([time.text for time in times]).value_counts()

8:30PM     4584
8:00PM     4443
3:00PM     2133
7:30PM     2075
2:30PM     1618
8:45PM     1470
2:00PM     1208
8:15PM      911
None        773
11:00AM     524
2:15PM      294
7:00PM      273
6:00PM      131
3:15PM      131
9:00PM      110
2:45PM      104
6:45PM      104
12:00PM      97
6:30PM       89
10:30AM      66
12:30PM      49
3:30PM       48
4:30PM       45
1:00PM       42
1:30PM       41
5:00PM       39
4:00PM       24
3:10PM       24
12:10PM      23
9:15PM       19
           ... 
12:00AM       6
11:30AM       6
7:30AM        6
4:45PM        6
8:03PM        5
6:15PM        5
10:00PM       4
2:20PM        4
8:15AM        4
8:30AM        3
12:15PM       3
3:00AM        2
10:15AM       2
12:45PM       2
7:50PM        2
8:25PM        2
7:03PM        2
8:00AM        2
9:10PM        2
9:45PM        1
8:10PM        1
1:15PM        1
3:20PM        1
2:00AM        1
1:50PM        1
6:03PM        1
8:36PM        1
2:50PM        1
1:45PM        1
2:30AM        1
Length: 70, dtype: int64

**Exercise 2.** How many total concerts did the New York Philharmonic perform in the 2014-15 season?

In Exercises 3-4, you will work with [APIXU](https://www.apixu.com/), an weather API. This API returns data in both JSON and XML formats. In these exercises, you should request the data to be returned in XML format.

Register with the website to obtain an API key. You will likely need to refer to [the API documentation here](https://www.apixu.com/doc/). If you run into unexpected errors, issue the HTTP request from your browser to make sure that the data is in the format you expect.

**Exercise 3.** Get the forecasted low (min) and high (max) temperatures (in Fahrenheit) for the next 7 days in San Luis Obispo. Make a graphic that displays this information.

**Exercise 4.** Get the hourly wind speed (in mph) for the past 7 days. (_Note:_ This will require making 7 HTTP requests to the API. Try to do it programmatically.) You should end up with $24 \times 7 = 168$ rows in your `DataFrame`. Make a plot of the wind speed as a function of time. What do you notice?

In [ ]:
# TYPE YOUR CODE HERE